In [40]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re 

In [41]:
url = 'https://www.congress.gov/search?q=%7B%22source%22%3A%22legislation%22%2C%22congress%22%3A118%7D'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)

In [42]:
result_headings = driver.find_elements_by_class_name('result-heading')

stage_items = driver.find_elements_by_css_selector('.result-item.result-tracker .stat_leg .selected')

# Extracting bills
bills = [heading.text.strip() for heading in result_headings if heading.text.strip()]

stages = [stage.text.strip() for stage in stage_items if stage.text.strip()]

driver.quit()


In [44]:
# Function to parse bill information
def parse_bill(bill):
    # Extract bill type (H.R., S., H.J.Res., etc.)
    bill_type = re.search(r'[A-Z.]+', bill).group()
    # Extract the number immediately following "H.R." and before "—"
    number = re.search(r'H\.R\.(\d+)(?=\s*—)', bill).group(1) if re.search(r'H\.R\.(\d+)(?=\s*—)', bill) else None
    
    if bill_type.startswith('H') and number:
        return 'house-bill', 'house', number
    elif bill_type.startswith('S'):
        return 'senate-bill', 'senate', number
    elif bill_type.startswith('H.J.Res'):
        return 'house-joint-resolution', 'house', number
    return None

# Create a DataFrame to hold the parsed bill information
new_data = []
for bill in bills:
    parsed_bill = parse_bill(bill)
    if parsed_bill:
        office, office_type, number = parsed_bill
        new_data.append({'original_name': bill, 'office': office, 'number': number})

new_bills_df = pd.DataFrame(new_data)

new_bills_df['stage'] = stages

In [45]:
new_bills_df

,original_name,office,number,stage
0,H.R.7154 — 118th Congress (2023-2024),house-bill,7154,Introduced
1,H.R.7153 — 118th Congress (2023-2024),house-bill,7153,Introduced
2,H.R.7152 — 118th Congress (2023-2024),house-bill,7152,Introduced
3,H.R.7151 — 118th Congress (2023-2024),house-bill,7151,Introduced
4,H.R.7150 — 118th Congress (2023-2024),house-bill,7150,Introduced
...,...,...,...,...
95,H.R.7059 — 118th Congress (2023-2024),house-bill,7059,Introduced
96,H.R.7058 — 118th Congress (2023-2024),house-bill,7058,Introduced
97,H.R.7057 — 118th Congress (2023-2024),house-bill,7057,Introduced
98,H.R.7056 — 118th Congress (2023-2024),house-bill,7056,Introduced
